In [256]:
from newsapi import NewsApiClient
import pandas as pd
from datetime import datetime, timedelta
# free newsapi only one month back 
newsapi = NewsApiClient(api_key="3f145b13615b45978d29b490c5c0f78a")
#qintitle='Germany   AND (crash OR bitcoin)'    allows only 3 words for search,want to search for something like "closes" and "closed" use close*  
articles = list()
# 
# We are greedy, take every news we can get until we reach the limit
measures = {'lockdown': [['lockdown','impos*'],['quarantin*','impose*'],["shutdown","shops"]],'schools' : [['schools','shut'],['schools','closed']]}

CountryList = ['german*','denmark','ital*','fran*','spain*',]
#CountryList = ['Spanien','Frankreich','Italien','Deutschland','Österreich','Dänemark']
#Object = ["Grenzen","Schulen",Ausgangsbeschränkung,"Ausgangssperre","Einreisebschränkung","Notstand"]
#ResponseList = ['Schlie*']
#Object = ['Schulen']
CountryDic = {}

def MatchHeadlines(Country,measure):
    CountryDic = {}
    land = Country
    _dict ={}
    for count,MeasureTouple in enumerate(measures[measure]):
        _measureCountDic = {}
        qInTitelVar = MeasureTouple[0]+' AND '+ MeasureTouple[1]+' AND ' + land   # Parameter
        articles = []
        for n in range(1,400):
            try:
                headlines = newsapi.get_everything( qintitle = qInTitelVar,
                language='en',                                         from_param='2020-02-24',page=n)
                articles = articles + headlines['articles']
            except: # We reached the page limit of our account
                break
        print(len(articles))
        dicvar = {str(count):{"ArticleList":articles,"Parameters": MeasureTouple}}
        _dict.update(dicvar)
        print(MeasureTouple,land)
    responseDic = {measure:_dict}
    return(responseDic)

In [275]:

qInTitelVar = 'morocco*'+' AND '+ 'school*' +' AND ' + 'shut*'   # Parameter
articles = []
for n in range(1,100):
    try:
        headlines = newsapi.get_everything( q = qInTitelVar,
                                                 from_param='2020-02-24',page=n)
        articles = articles + headlines['articles']
    except: # We reached the page limit of our account
        break
print(len(articles))

print(articles)

100
[{'description': "Virus restrictions tighten, disrupting daily life, worship CTV News Spain's streets turn eerily quiet following coronavirus lockdown The Guardian Spain imposes Italy-style lockdown in bid to contain coronavirus Al Jazeera English Spain announces national lock…", 'publishedAt': '2020-03-15T18:37:00Z', 'content': 'ROME -- \r\nDaily life came to a grinding halt around much of the world Sunday amid new travel restrictions, border shutdowns and sweeping closures of restaurants and bars aimed at slowing the spread of the coronavirus pandemic.\r\nThe numbers of cases and deaths… [+8955 chars]', 'author': 'Joseph Wilson and Geir Moulson', 'url': 'https://www.ctvnews.ca/health/coronavirus/virus-brings-new-travel-restrictions-lockdowns-and-closures-1.4853740', 'source': {'name': 'Ctvnews.ca', 'id': None}, 'urlToImage': 'https://www.ctvnews.ca/polopoly_fs/1.4853743.1584270508!/httpImage/image.jpg_gen/derivatives/landscape_620/image.jpg', 'title': 'Virus restrictions tighten,

In [186]:
dicts = {}
print(dicts.get("lol"))

None


In [255]:
cab = {"france":MatchHeadlines('france','lockdown')}
if cab.get("germany") == None:
    cab["germany"] = MatchHeadlines('germany','schools')
else:
    cab["germany"].update(MatchHeadlines('germany','schools'))
cab["germany"].update(MatchHeadlines('germany','lockdown'))

0
['lockdown', 'impos*'] france
0
['quarantin*', 'impose*'] france
0
['shutdown', 'shops'] france
0
['schools', 'shut'] germany
0
['schools', 'closed'] germany
0
['lockdown', 'impos*'] germany
0
['quarantin*', 'impose*'] germany
0
['shutdown', 'shops'] germany


In [221]:
cab.get("germany").get("schools").get("1").get('ArticleList')

[{'author': None,
  'content': 'All but a handful of the Pentagons 66 schools across Europe will close Monday to students as a precautionary measure to help curb the spread of COVID-19, school and military leaders announced Friday.\r\nThe new closures apply to all Department of Defense Educat… [+1127 chars]',
  'description': 'DODEA schools in Germany, Belgium and the Netherlands will be closed beginning Monday due to coronavirus concerns, according to Installation Management Command Directorate-Europe.',
  'publishedAt': '2020-03-13T12:38:00Z',
  'source': {'id': None, 'name': 'Stripes.com'},
  'title': 'DODEA Europe closes schools in Germany, Belgium and the Netherlands due to coronavirus concerns',
  'url': 'https://www.stripes.com/news/europe/dodea-closes-schools-in-germany-belgium-the-netherlands-and-spain-due-to-coronavirus-concerns-1.622290?utm_medium=feed&utm_source=feedburner&utm_campaign=Feed%3A+starsandstripes%2Fgeneral+%28Stars+and+Stripes%29',
  'urlToImage': 'https://www.

In [178]:
cab.get("germany").keys()

dict_keys(['schools', 'lockdown'])

In [146]:
cab["germany"].keys()

dict_keys(['schools', 'lockdown'])

In [103]:
CountryDic2['Austria'].update({'lockdown': {'0': {'ArticleList': [],
    'Parameters': ['lockdown', 'imposed']},
   '1': {'ArticleList': [], 'Parameters': ['quarantine', 'impose*']}}})

In [104]:
CountryDic2

{'Austria': {'lockdown': {'0': {'ArticleList': [],
    'Parameters': ['lockdown', 'imposed']},
   '1': {'ArticleList': [], 'Parameters': ['quarantine', 'impose*']}},
  'schools': {'0': {'ArticleList': [], 'Parameters': ['schools', 'shut*']},
   '1': {'ArticleList': [], 'Parameters': ['schools', 'close*']}}},
 'denmark': {'lockdown': {'0': {'ArticleList': [],
    'Parameters': ['lockdown', 'imposed']},
   '1': {'ArticleList': [], 'Parameters': ['quarantine', 'impose*']}}},
 'germany': {'lockdown': {'0': {'ArticleList': [],
    'Parameters': ['lockdown', 'imposed']},
   '1': {'ArticleList': [], 'Parameters': ['quarantine', 'impose*']}}}}

In [13]:
def LikelyDayOfGovResponse(ArticleVar):
    df = pd.DataFrame(ArticleVar)
    df['publishedAt'] = pd.to_datetime(df['publishedAt'])
    test = df.resample('D', on='publishedAt').count()
    mask = (df['publishedAt'] >= test['source'].argmax()) & (df['publishedAt'] < test['source'].argmax() + timedelta(days = 1))
    pd.set_option('display.max_colwidth', 20000)
    df['source'] = df['source'].apply(lambda r : r['name'])
    dfArgmax = df[mask].drop(columns=['author','urlToImage','content'])
    return(dfArgmax)

In [9]:
def LikelyDayOfGovResponse(ArticleVar):
    df = pd.DataFrame(ArticleVar)
    df['publishedAt'] = pd.to_datetime(df['publishedAt'])
    test = df.resample('D', on='publishedAt').count()
    mask = (df['publishedAt'] >= test['source'].argmax()) & (df['publishedAt'] < test['source'].argmax() + timedelta(days = 1))
    pd.set_option('display.max_colwidth', 20000)
    df['source'] = df['source'].apply(lambda r : r['name'])
    dfArgmax = df[mask].drop(columns=['author','urlToImage','content'])
    return(dfArgmax)

In [10]:
artVar = CountryDic.get('Frankreich').get('ArticleList')
ResponseDf = LikelyDayOfGovResponse(artVar)
ResponseDf["MatchParameter"] = CountryDic.get('Frankreich').get("Parameters")
ResponseDf

C:\Users\eddy-\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


,description,publishedAt,source,title,url,MatchParameter
0,"In Frankreich bleiben ab Montag alle Schulen, Universitäten und Kindergärten geschlossen. Präsident Macron sagt, das Land stehe erst am Anfang einer Pandemie.",2020-03-12 20:44:34,Faz.net,"Frankreich lässt Schulen, Unis, Kindergärten schließen",https://www.faz.net/aktuell/politik/ausland/frankreich-laesst-schulen-unis-kindergaerten-schliessen-16677096.html,Schulen AND Schlie* AND Frankreich
1,Der französische Staatspräsident Emmanuel Macron hat drastische Maßnahmen zur Eindämmung des Coronavirus verordnet. Alle Schulen und Kitas in Frankreich bleiben geschlossen. Außerdem rief er alle Franzosen zu weiteren Handlungen auf.,2020-03-12 19:57:45,Welt.de,"Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag",https://www.welt.de/vermischtes/article206526735/Coronavirus-Frankreich-schliesst-alle-Schulen-und-Kitas.html,Schulen AND Schlie* AND Frankreich
2,None,2020-03-12 19:38:10,Orf.at,Frankreich schließt bis auf Weiteres alle Schulen,https://orf.at/stories/3157628/,Schulen AND Schlie* AND Frankreich
3,Merkel: Wegen Coronavirus auf Sozialkontakte weitgehend verzichten +++ Bund und Länder: Ab Montag alle planbaren Operationen verschieben +++ Dow Jones erleidet schwersten Verlust seit 33 Jahren +++ Die wichtigsten überregionalen Entwicklungen im Newsblog.,2020-03-12 19:22:00,Berliner-zeitung.de,"Coronavirus: Frankreich schließt alle Schulen, Hochschulen und Kitas - Berliner Zeitung",https://www.berliner-zeitung.de/gesundheit-oekologie/zahl-der-corona-toten-in-deutschland-steigt-auf-sechs-li.5643,Schulen AND Schlie* AND Frankreich
4,"Coronavirus: Kampf gegen Pandemie – Frankreich schließt alle Schulen t-online.de Macron kündigt massive Hilfen an:Frankreich schließt alle Schulen n-tv NACHRICHTEN ""Mehr zum Thema"" ansehen in Google News",2020-03-12 20:26:00,T-online.de,Coronavirus: Kampf gegen Pandemie – Frankreich schließt alle Schulen - t-online.de,https://www.t-online.de/nachrichten/panorama/id_87512202/coronavirus-kampf-gegen-pandemie-frankreich-schliesst-alle-schulen.html,Schulen AND Schlie* AND Frankreich
5,"Im Kampf gegen die Coronavirus-Krise hat Frankreichs Präsident Emmanuel Macron die Schließung aller Schulen, Hochschulen und Kindertagesstätten im Land angekündigt.",2020-03-12 19:36:03,Stern.de,"Macron: Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag",https://www.stern.de/news/macron--frankreich-schliesst-alle-schulen--hochschulen-und-kitas-ab-montag-9180158.html,Schulen AND Schlie* AND Frankreich
6,"Das Coronavirus breitet sich weiter aus, Veranstaltungen werden reihenweise abgesagt, das öffentliche Leben kommt mehr und mehr zum Erliegen. In Deutschland gibt es derzeit fünf Todesfälle und mehr als 1.500 bestätigte Corona-Infizierte. Weltweit sind es bere…",2020-03-12 19:23:40,Focus,"Corona-Krise - Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag - weiterer Todesfall in Heinsberg",https://www.focus.de/gesundheit/news/corona-krise-frankreich-schliesst-alle-schulen-hochschulen-und-kitas-ab-montag-weiterer-todesfall-in-heinsberg_id_11765515.html,Schulen AND Schlie* AND Frankreich
8,Europäische Zentralbank verkündet Hilfspaket + Halle schließt Schulen und Kitas + Coronavirus erreicht den Deutschen Bundestag + Dax stürzt ab + Der Newsblog.,2020-03-12 19:21:36,Der Tagesspiegel,"Frankreich schließt ab Montag alle Schulen, Unis und Kitas",https://www.tagesspiegel.de/wissen/kampf-gegen-das-coronavirus-frankreich-schliesst-ab-montag-alle-schulen-unis-und-kitas/25560996.html,Schulen AND Schlie* AND Frankreich
9,Mit drastischen Worten bereitet Präsident Macron die Franzosen auf Einschnitte im öffentlichen Leben vor. Infolge des Corona-Virus befinde sich das Land in der schlimmsten Gesundheitskrise seit einem Jahrhundert. Bis auf weiteres schließen landesweit alle Bil…,2020-03-12 19:58:43,N-tv.de,Macron kündigt massive Hilfen an: Frankreich schließt alle Schulen,https://www.n-tv.de/panorama/Frankreich-schliesst-alle-Schulen-article21

In [612]:
ResponseDf

,description,publishedAt,source,title,url,MatchParameter
0,"In Frankreich bleiben ab Montag alle Schulen, Universitäten und Kindergärten geschlossen. Präsident Macron sagt, das Land stehe erst am Anfang einer Pandemie.",2020-03-12 20:44:34,Faz.net,"Frankreich lässt Schulen, Unis, Kindergärten schließen",https://www.faz.net/aktuell/politik/ausland/frankreich-laesst-schulen-unis-kindergaerten-schliessen-16677096.html,Schulen AND Schlie* Dänemark
1,Der französische Staatspräsident Emmanuel Macron hat drastische Maßnahmen zur Eindämmung des Coronavirus verordnet. Alle Schulen und Kitas in Frankreich bleiben geschlossen. Außerdem rief er alle Franzosen zu weiteren Handlungen auf.,2020-03-12 19:57:45,Welt.de,"Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag",https://www.welt.de/vermischtes/article206526735/Coronavirus-Frankreich-schliesst-alle-Schulen-und-Kitas.html,Schulen AND Schlie* Dänemark
2,None,2020-03-12 19:38:10,Orf.at,Frankreich schließt bis auf Weiteres alle Schulen,https://orf.at/stories/3157628/,Schulen AND Schlie* Dänemark
3,Merkel: Wegen Coronavirus auf Sozialkontakte weitgehend verzichten +++ Bund und Länder: Ab Montag alle planbaren Operationen verschieben +++ Dow Jones erleidet schwersten Verlust seit 33 Jahren +++ Die wichtigsten überregionalen Entwicklungen im Newsblog.,2020-03-12 19:22:00,Berliner-zeitung.de,"Coronavirus: Frankreich schließt alle Schulen, Hochschulen und Kitas - Berliner Zeitung",https://www.berliner-zeitung.de/gesundheit-oekologie/zahl-der-corona-toten-in-deutschland-steigt-auf-sechs-li.5643,Schulen AND Schlie* Dänemark
4,"Coronavirus: Kampf gegen Pandemie – Frankreich schließt alle Schulen t-online.de Macron kündigt massive Hilfen an:Frankreich schließt alle Schulen n-tv NACHRICHTEN ""Mehr zum Thema"" ansehen in Google News",2020-03-12 20:26:00,T-online.de,Coronavirus: Kampf gegen Pandemie – Frankreich schließt alle Schulen - t-online.de,https://www.t-online.de/nachrichten/panorama/id_87512202/coronavirus-kampf-gegen-pandemie-frankreich-schliesst-alle-schulen.html,Schulen AND Schlie* Dänemark
5,"Im Kampf gegen die Coronavirus-Krise hat Frankreichs Präsident Emmanuel Macron die Schließung aller Schulen, Hochschulen und Kindertagesstätten im Land angekündigt.",2020-03-12 19:36:03,Stern.de,"Macron: Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag",https://www.stern.de/news/macron--frankreich-schliesst-alle-schulen--hochschulen-und-kitas-ab-montag-9180158.html,Schulen AND Schlie* Dänemark
6,"Das Coronavirus breitet sich weiter aus, Veranstaltungen werden reihenweise abgesagt, das öffentliche Leben kommt mehr und mehr zum Erliegen. In Deutschland gibt es derzeit fünf Todesfälle und mehr als 1.500 bestätigte Corona-Infizierte. Weltweit sind es bere…",2020-03-12 19:23:40,Focus,"Corona-Krise - Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag - weiterer Todesfall in Heinsberg",https://www.focus.de/gesundheit/news/corona-krise-frankreich-schliesst-alle-schulen-hochschulen-und-kitas-ab-montag-weiterer-todesfall-in-heinsberg_id_11765515.html,Schulen AND Schlie* Dänemark
8,Europäische Zentralbank verkündet Hilfspaket + Halle schließt Schulen und Kitas + Coronavirus erreicht den Deutschen Bundestag + Dax stürzt ab + Der Newsblog.,2020-03-12 19:21:36,Der Tagesspiegel,"Frankreich schließt ab Montag alle Schulen, Unis und Kitas",https://www.tagesspiegel.de/wissen/kampf-gegen-das-coronavirus-frankreich-schliesst-ab-montag-alle-schulen-unis-und-kitas/25560996.html,Schulen AND Schlie* Dänemark
9,Mit drastischen Worten bereitet Präsident Macron die Franzosen auf Einschnitte im öffentlichen Leben vor. Infolge des Corona-Virus befinde sich das Land in der schlimmsten Gesundheitskrise seit einem Jahrhundert. Bis auf weiteres schließen landesweit alle Bil…,2020-03-12 19:58:43,N-tv.de,Macron kündigt massive Hilfen an: Frankreich schließt alle Schulen,https://www.n-tv.de/panorama/Frankreich-schliesst-alle-Schulen-article21637864.html,Schulen AND Schlie* Dänemark
10,"Weg